## Photometry on MIRI images

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import warnings
import subprocess
import json
import matplotlib.pyplot as plt

from PIL import Image

from matplotlib.patches import Ellipse

from astropy.io import fits
from astropy.wcs import WCS, FITSFixedWarning
from astropy.table import Table
import astropy.units as u
from astropy.stats import SigmaClip
from astropy.visualization import ZScaleInterval

from photutils.aperture import EllipticalAperture, EllipticalAnnulus, RectangularAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

warnings.simplefilter("ignore", category=FITSFixedWarning)


cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"


# Section to obtain modified apertures

Let's inspect Amirs table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
#print(table[:5])
table.info()
print(table.columns)

Now let's choose a galaxy, read its aperture data and overplot it!

In [ ]:
def load_cutout(file_path, index=1):
    """Loads a FITS cutout file and extracts the data, header, and WCS."""
    try:
        with fits.open(file_path) as hdu:
            data = hdu[index].data
            header = hdu[index].header
            wcs = WCS(header)
        return data, wcs
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None, None

def adjust_aperture(galaxy_id, filter, survey, obs, output_folder, save_plot=False):
    
    # --- Load the FITS table ---
    #table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'
    #aperture_table = Table.read(table_path)
    table_path =  '/home/bpc/University/master/Red_Cardinal/aperture_table.csv'
    df = pd.read_csv(table_path)
    
    # --- Select the galaxy by ID ---
    #row = aperture_table[aperture_table['ID'] == galaxy_id][0]
    galaxy_id = int(galaxy_id)
    row = df[df['ID'] == galaxy_id].iloc[0]

    # --- Read in rotation angle of MIRI FITS file ---
    angle_file = '/home/bpc/University/master/Red_Cardinal/rotation_angles.json'
    with open(angle_file, "r") as f:
        angles = json.load(f)
    angle = angles[f"angle_{survey}{obs}"]
    
    # --- Read WCS from NIRCam image ---
    nircam_path = f"/home/bpc/University/master/Red_Cardinal/NIRCam/F444W_cutouts/{galaxy_id}_F444W_cutout.fits"
    nircam_data, nircam_wcs = load_cutout(nircam_path)

    # --- Convert NIRCam pixel coordinates to sky ---
    sky_coord = nircam_wcs.pixel_to_world(row['Apr_Xcenter'], row['Apr_Ycenter'])
    
    # --- Open MIRI cutout image ---
    miri_path = f"/home/bpc/University/master/Red_Cardinal/cutouts_phot/{galaxy_id}_{filter}_cutout_{survey}{obs}.fits"
    miri_data, miri_wcs = load_cutout(miri_path)

    # --- Convert sky coords to MIRI pixel coordinates ---
    miri_x, miri_y = miri_wcs.world_to_pixel(sky_coord)

    # --- Create elliptical region in MIRI pixel space ---
    nircam_scale = 0.03    # arcsec/pixel
    miri_scale = 0.11092  # arcsec per pixel
    
    # arcsec/pixel
    scale_factor = nircam_scale / miri_scale
    
    # --- Specify parameters for the ellips ---
    width = row['Apr_A'] * scale_factor
    height = row['Apr_B'] * scale_factor
    theta = -row['Apr_Theta']
    theta_new = ((theta - angle) % 180) * u.deg
    
    # --- Create region file and check if folder exists ---
    os.makedirs(output_folder, exist_ok=True)
    reg_file = os.path.join(output_folder, f'regions/{galaxy_id}_{survey}{obs}_aperture.reg') 
    
    # --- Write to DS9-compatible region file ---
    with open(reg_file, "w") as fh:
        fh.write("# Region file format: DS9 version 4.1\n")
        fh.write("global color=red dashlist=8 3 width=2 font=\"helvetica 10 normal\" "
                "select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\n")
        fh.write("image\n")
        fh.write(f"ellipse({miri_x:.2f},{miri_y:.2f},{width:.2f},{height:.2f},{theta_new:.2f})\n")
    
    if save_plot:
        
        # --- Clean and prepare the MIRI data for plotting ---
        miri_clean = np.copy(miri_data)
        finite_vals = miri_clean[np.isfinite(miri_clean)].flatten()
        
        # Sort and get lowest 80% values
        sorted_vals = np.sort(finite_vals)
        cutoff_index = int(0.8 * len(sorted_vals))
        background_vals = sorted_vals[:cutoff_index]
        background_mean = np.mean(background_vals)

        # Replace NaNs or infs with background mean
        miri_clean[~np.isfinite(miri_clean)] = background_mean

        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(miri_clean, origin='lower', cmap='gray', vmin=np.percentile(miri_clean, 5), vmax=np.percentile(miri_clean, 99))

        # Original ellipse (without rotation correction)
        ellipse_original = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta,  # Just the original θ!
            edgecolor='red',
            facecolor='none',
            lw=2,
            label='Original Ellipse'
        )
        ax.add_patch(ellipse_original)

        ellipse = Ellipse(
            xy=(miri_x, miri_y),
            width=width,
            height=height,
            angle=theta_new.to_value(u.deg),
            edgecolor='blue',
            linestyle='--',  # maybe dashed to differentiate
            facecolor='none',
            lw=2,
            label='Rotated Ellipse'
        )
        ax.add_patch(ellipse)
        
        ax.set_title(f"Galaxy {galaxy_id} - {filter} ({survey}{obs})")
        ax.set_xlim(miri_x - 30, miri_x + 30)
        ax.set_ylim(miri_y - 30, miri_y + 30)
        ax.legend(loc='upper right')
        
        # Save figure
        png_path = os.path.join(output_folder, f'masks/{galaxy_id}_{survey}{obs}_aperture_overlay.png')
        plt.savefig(png_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        
    # Collect modified aperture data
    aperture_info = {          
    'Apr_A': width,            # Rescaled aperture
    'Apr_B': height,
    'Apr_Xcenter': miri_x,
    'Apr_Ycenter': miri_y,
    'Apr_Theta': theta_new.to_value(u.deg),
    'ID': galaxy_id
    }
    
    return aperture_info
    

Now let's try and call the function:

In [ ]:
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

# Get all FITS file paths
fits_files = glob.glob(os.path.join(cutout_dir, '*.fits'))

# Get the basenames of the FITS files
fits_fnames = [os.path.basename(f) for f in fits_files]

adjusted_apertures = []

for fname in fits_fnames:
    id = fname.split('_')[0]
    filter = fname.split('_')[1]
    if filter == 'F1800W': continue
    survey_obs = fname.split('_')[3]
    if '003' in survey_obs:
        survey = 'primer'
        obs = '003'
    elif '004' in survey_obs:
        survey = 'primer'
        obs = '004'
    elif '1' in survey_obs:
        survey = 'cweb'
        obs = '1'
    elif '2' in survey_obs:
        survey = 'cweb'
        obs = '2'
    else:
        print(f"Unknown survey and/or observation number for galaxy {id}:\n")
        print(survey_obs)
    
    # Call and collect results
    result = adjust_aperture(id, filter, survey, obs, phot_dir, save_plot=False)
    if result:
        adjusted_apertures.append(result)

# After loop: create a DataFrame
df_apertures = pd.DataFrame(adjusted_apertures)

df_path = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_rot.csv'

# (optional) Save to CSV or integrate into photometry table
df_apertures.to_csv(df_path, index=False)

Now we can easily open any given FITS file with its corresponding ellipse region

In [ ]:
# --- Launch DS9 with the MIRI cutout and the overplotted aperture ---
region_dir = "/home/bpc/University/master/Red_Cardinal/photometry/regions/"
cutout_dir = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
phot_dir = "/home/bpc/University/master/Red_Cardinal/photometry/"

id = '10245'
filter = 'F770W'
survey_obs = 'primer004'
cutout_path = os.path.join(cutout_dir, f'{id}_{filter}_cutout_{survey_obs}.fits')
reg_path = os.path.join(region_dir, f'{id}_{survey_obs}_aperture.reg')
subprocess.run(["ds9", cutout_path, "-regions", reg_path])


Let's check the table:

In [ ]:
table_path =  '/home/bpc/University/master/Red_Cardinal/Flux_Aperture_PSFMatched_AperCorr_old.fits'

table = Table.read(table_path)
print(table[:5])
table.info()
print(table['Image_Err'].shape)

# This is where we do most of the work

# Section to perform the actual photometry

First, let's set some parameters straight:

pixscale_arcsec = 0.11092  # arcsec per pixel

pix_area_sr = 2.89208962133982e-13  # from MIRI header


In [ ]:
def estimate_background(galaxy_id, filter_name, image_data, aperture_params, sigma=3.0, 
                        annulus_factor=3.0, fig_path=None):
    """
    Estimate background using a global 2D plane fit, then extract statistics from 
    an elliptical annulus.
    
    Parameters
    ----------
    galaxy_id : str
        The ID of the galaxy
    filter_name : str
        The band which is being observed
    image_data : ndarray
        The 2D image data
    aperture_params : dict
        Dictionary containing aperture parameters (x_center, y_center, a, b, theta)
    sigma : float
        Sigma clipping threshold
    annulus_factor : float
        Factor by which to scale the inner ellipse to create the outer ellipse
    visualise : bool, optional
        If True, display visualisation plots
        
    Returns
    -------
    - background_plane : ndarray
        2D background model
    - background_median : float
        median background value within the annulus
    - background_std : float
        standard deviation of background model within the annulus (excluding clipped data)
    - background_region_mask : ndarray
        boolean mask showing the region used for background stats

    """
    
    x_center = aperture_params['x_center']
    y_center = aperture_params['y_center']
    a = aperture_params['a']
    b = aperture_params['b']
    theta = aperture_params['theta']    # in radians
    
    # Create source aperture
    source_aperture = EllipticalAperture(
        positions=(x_center, y_center),
        a=a,
        b=b,
        theta=theta
    )
    
    # Create mask for the source
    source_mask = source_aperture.to_mask(method='center').to_image(image_data.shape)
    source_mask_bool = source_mask.astype(bool)
    
    # Apply initial source mask to the data
    masked_data = np.copy(image_data)
    masked_data[source_mask_bool] = np.nan
    
    # Apply sigma clipping to the remaining background
    sigma_clip = SigmaClip(sigma=sigma)
    clipped_data = sigma_clip(masked_data)
    
    # Create a mask for sigma-clipped pixels (clipped_data.mask is True for clipped values)
    sigma_clipped_mask = clipped_data.mask if hasattr(clipped_data, 'mask') else np.isnan(clipped_data)
    
    # Create a global mask for all valid background pixels (not in source, not sigma-clipped)
    global_mask = ~source_mask_bool & ~np.isnan(image_data) & ~sigma_clipped_mask
    
    # Define indices for the full image
    y, x = np.indices(image_data.shape)
    
    # Extract coordinates and values of all valid background pixels for global fitting
    x_vals = x[global_mask].flatten()
    y_vals = y[global_mask].flatten()
    z_vals = image_data[global_mask].flatten()
    
    # Check if we have enough pixels for fitting
    if len(z_vals) < 3:
        raise ValueError("Not enough background pixels for fitting. Try adjusting parameters.")
    
    # Fit a 2D plane (ax + by + c) to all valid background pixels
    A = np.vstack([x_vals, y_vals, np.ones_like(x_vals)]).T
    coeffs, residuals, rank, s = np.linalg.lstsq(A, z_vals, rcond=None)
    alpha, beta, gamma = coeffs
    
    # Create the 2D background plane for the entire image
    background_plane = alpha * x + beta * y + gamma
    
    # Define the background region
    region_name = "Annulus"
    
    # Set minimum and maximum sizes for the annulus
    min_pixels = 300  # Minimum number of pixels in the annulus for reliable background estimation
    max_annulus_size = 35  # Roughly half of the 75x75 image size for large apertures
    step_factor = 0.15  # Step size for expanding the annulus
    max_attempts = 10
    attempt = 0
    
    # Start with a slightly larger annulus than the source aperture
    a_in = a * 2
    b_in = b * 2
    a_out = a_in * annulus_factor
    b_out = b_in * annulus_factor
    
    if galaxy_id in ['12020', '17669', '7136']:
        a_in *= 1.2
        b_in *= 1.2
        a_out *= 1.3
        b_out *= 1.3
    if galaxy_id in ['9871', '11136', '11137', '11494', '12340', '12717', '17793', '16874', '17517', '20397']:
        a_out *= 0.7
        b_out *= 0.7
    
    # Adjust the outer size until the annulus contains enough pixels
    while attempt < max_attempts:
        # Create the annulus
        annulus = EllipticalAnnulus(
            positions=(x_center, y_center),
            a_in=a_in,
            b_in=b_in,
            a_out=min(a_out, max_annulus_size),
            b_out=min(b_out, max_annulus_size),
            theta=theta
        )
        
        # Create mask for the annulus region
        annulus_mask = annulus.to_mask(method='center').to_image(image_data.shape)
        background_region_mask = annulus_mask.astype(bool) & ~np.isnan(image_data)
        pixel_count = np.sum(background_region_mask)

        # If the annulus has enough pixels, break the loop
        if pixel_count >= min_pixels:
            break
        
        # Expand the annulus for the next attempt
        a_out += a_in * step_factor
        b_out += b_in * step_factor
        attempt += 1
        
    
    # Gather all the pixels within the background region that were NOT sigma-clipped
    bkg_region_valid_pixels = ~sigma_clipped_mask & background_region_mask
    residual_data = image_data - background_plane
    num_valid_pixels = np.sum(bkg_region_valid_pixels)
    
    # Calculate background statistics
    background_residuals = residual_data[bkg_region_valid_pixels]
    background_std = np.std(background_residuals) * np.sqrt(num_valid_pixels)
    
    # Extract background plane values for the background region and calculate the median
    bkg_plane_values = background_plane[background_region_mask]
    background_median = np.median(bkg_plane_values)
    
    # Print background statistics
    print(f"Background Statistics:")
    print(f"  Global 2D Plane coefficients: a={alpha:.6e}, b={beta:.6e}, c={gamma:.6f}")
    print(f"  {region_name} region background median: {background_median:.6f}")
    print(f"  {region_name} region background std dev: {background_std:.6f}")
    
    # Create a mask visualisation
    mask_vis = np.zeros_like(image_data, dtype=int)
    mask_vis[~sigma_clipped_mask] = 1  # Pixels excluded by sigma clipping
    mask_vis[background_region_mask] = 2  # Pixels in the annulus/rectangle
    mask_vis[source_mask_bool] = 3  # Source pixels
    
    # Store visualization data
    vis_data = {
        'galaxy_id': galaxy_id,
        'filter': filter_name,
        'original_data': image_data,
        'background_plane': background_plane,
        'background_subtracted': residual_data,
        'mask_vis': mask_vis,
        'sigma_clipped_mask': sigma_clipped_mask,
        'background_region_mask': background_region_mask,
        'source_mask': source_mask_bool,
        'aperture_params': aperture_params,
        'a_in': a_in,
        'b_in': b_in,
        'a_out': a_out,
        'b_out': b_out,
        'sigma': sigma,
        'region_name': region_name,
        'coeffs': (alpha, beta, gamma)
    }
    
    # If requested, visualize the results
    if fig_path:
        visualise_background(vis_data, fig_path=fig_path)
    
    return background_median, background_std



def visualise_background(vis_data, fig_path=None):
    """
    Create visualisations from the background estimation data.
    
    Parameters
    ----------
    vis_data : dict
        Dictionary containing all data needed for visualisation
    fig_path : str, optional
        Path to save the visualisation figure
    """
    # Extract data from the dictionary
    image_data = vis_data['original_data']
    background_plane = vis_data['background_plane']
    background_subtracted = vis_data['background_subtracted']
    mask_vis = vis_data['mask_vis']
    aperture_params = vis_data['aperture_params']
    sigma = vis_data['sigma']
    region_name = vis_data['region_name']
    galaxy_id = vis_data['galaxy_id']
    filter = vis_data['filter']
    
    # Create aperture objects for plotting
    x_center = aperture_params['x_center']
    y_center = aperture_params['y_center']
    a = aperture_params['a']
    b = aperture_params['b']
    theta = aperture_params['theta']
    
    source_aperture = EllipticalAperture(
        positions=(x_center, y_center),
        a=a,
        b=b,
        theta=theta
    )
    
    # Create visualisations
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Original data with aperture
    vmin = np.nanpercentile(image_data, 5)
    vmax = np.nanpercentile(image_data, 95)
    
    im0 = axes[0, 0].imshow(image_data, origin='lower', cmap='magma', vmin=vmin, vmax=vmax)
    plt.colorbar(im0, ax=axes[0, 0], label='Flux [MJy/(sr pixel)]')
    
    # Plot the source aperture
    source_aperture.plot(ax=axes[0, 0], color='red', lw=1.5)
    
    # Plot the background region
    annulus = EllipticalAnnulus(
        positions=(x_center, y_center),
        a_in  = vis_data['a_in'],
        b_in  = vis_data['b_in'],
        a_out = vis_data['a_out'],
        b_out = vis_data['b_out'],
        theta = theta
    )
    annulus.plot(ax=axes[0, 0], color='white', lw=1.5)
        
    axes[0, 0].set_title("Original Data with Aperture and Annulus")
    
    # Background-subtracted data
    vmin2 = np.nanpercentile(background_subtracted, 5)
    vmax2 = np.nanpercentile(background_subtracted, 95)
    
    im1 = axes[0, 1].imshow(background_subtracted, origin='lower', cmap='magma', vmin=vmin2, vmax=vmax2)
    plt.colorbar(im1, ax=axes[0, 1], label='Background-subtracted Flux [MJy/(sr pixel)]')
    source_aperture.plot(ax=axes[0, 1], color='red', lw=1.5)
    axes[0, 1].set_title("Background-subtracted Data with Aperture")
    
    # Global 2D background plane
    im2 = axes[1, 0].imshow(background_plane, origin='lower', cmap='viridis')
    plt.colorbar(im2, ax=axes[1, 0], label='Background Flux [MJy/(sr pixel)')
    axes[1, 0].set_title("Global 2D Background Plane")
        
    # Mask visualisation
    cmap = plt.cm.get_cmap('viridis', 4)
    im3 = axes[1, 1].imshow(mask_vis, origin='lower', cmap=cmap, vmin=-0.5, vmax=3.5)
    cbar = plt.colorbar(im3, ax=axes[1, 1], ticks=[0, 1, 2, 3])
    cbar.set_ticklabels([f'Excluded\n(σ={sigma})', 'Used for fitting', 
                         f'{region_name} region', 'Source'])
    axes[1, 1].set_title("Pixel Masks")
    
    fig.suptitle(f'{filter}', fontsize=18)#, fontweight='bold')
    plt.tight_layout()
    
    if fig_path:
        os.makedirs(fig_path, exist_ok=True)
        if filter == 'F1800W':
            filepath = os.path.join(fig_path, f'{galaxy_id}_{filter}.png')
        else: 
            filepath = os.path.join(fig_path, f'{galaxy_id}.png')
        plt.savefig(filepath, dpi=150)
        plt.close(fig)

def get_psf(filter_name, psf_dir='/home/bpc/University/master/Red_Cardinal/WebbPSF/'):
    """
    Read MIRI PSF file for the specified filter.
    
    Parameters
    ----------
    filter_name : str
        Name of the filter
    psf_dir : str
        Directory containing PSF files
        
    Returns
    -------
    psf_data : ndarray
        PSF data
    """
    psf_file = os.path.join(psf_dir, f'PSF_MIRI_{filter_name}.fits')    
    with fits.open(psf_file) as psf:
        return psf[0].data

def get_aperture_params(galaxy_id, aperture_table):
    """
    Retrieve aperture parameters from the CSV table.
    
    Parameters
    ----------
    galaxy_id : str
        ID of the galaxy
    aperture_table : str
        Path to CSV table with aperture parameters
        
    Returns
    -------
    dict
        Dictionary with aperture parameters
    """
    df = pd.read_csv(aperture_table)
    row = df[df['ID'] == int(galaxy_id)].iloc[0]
    
    return {
        'x_center': row['Apr_Xcenter'],
        'y_center': row['Apr_Ycenter'], 
        'a': row['Apr_A'] / 2,  # Converting diameter to radius
        'b': row['Apr_B'] / 2,  # Converting diameter to radius
        'theta': (row['Apr_Theta'] * u.deg).to_value(u.rad)  # Convert to radians
    }

def calculate_aperture_correction(psf_data, aperture_params):
    """
    Calculate aperture correction factor for given PSF and aperture.
    
    Parameters
    ----------
    psf_data : ndarray
        PSF data
    aperture_params : dict
        Aperture parameters
        
    Returns
    -------
    correction_factor : float
        Aperture correction factor
    """
    aperture = EllipticalAperture(
        positions=(psf_data.shape[1] / 2, psf_data.shape[0] / 2),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )
    total_flux = np.sum(psf_data)
    phot_table = aperture_photometry(psf_data, aperture)
    flux_in_aperture = phot_table['aperture_sum'][0]
    return total_flux / flux_in_aperture

def measure_flux(image_data, error_map, background_median, background_std, aperture_params):
    """
    Calculate flux and uncertainty from aperture photometry.
    
    Parameters
    ----------
    image_data : ndarray
        Image data
    background_median : float
        Median background level
    error_map : ndarray
        Error map data
    background_std : float
        Standard deviation of background
    aperture_params : dict
        Aperture parameters
        
    Returns
    -------
    dict
        Dictionary with flux measurements and uncertainties
    """
    # Subtract the median background value within the annulus from the data
    data_bkgsub = image_data - background_median
    
    # Create aperture
    aperture = EllipticalAperture(
        positions=(aperture_params['x_center'], aperture_params['y_center']),
        a=aperture_params['a'],
        b=aperture_params['b'],
        theta=aperture_params['theta']
    )
    
    # Sum flux within the aperture - baclground subtracted data
    phot_table = aperture_photometry(data_bkgsub, aperture, method='exact')
    flux = phot_table['aperture_sum'][0]    # in MJy/sr
    
    # Mask for aperture
    aperture_mask = aperture.to_mask(method='exact')
    mask = aperture_mask.to_image(data_bkgsub.shape)
    
    # Flux uncertainty from ERR extension
    image_errors = error_map * mask
    sum_image_errors = np.sqrt(np.sum(image_errors**2))
    
    # Number of pixels within the aperture
    n_pix = aperture.area
    
    # To obtain the background flux within the aperture we multiply the median background within the annulus
    # by the number of pixels within the aperture    
    background_flux = n_pix * background_median
    
    # Total flux uncertainty
    total_flux_error = np.sqrt(sum_image_errors**2 + background_std**2)
    
    # Median error of the error map within the aperture
    median_error = np.median(error_map[mask>0])    
    
    # Convert everything to from MJy/sr to Jy
    miri_scale = 0.11092  # arcsec per pixel
    miri_scale_rad = miri_scale / 206265
    omega_pix = miri_scale_rad**2
    conversion_factor = 1e6 * omega_pix
    
    # Now everything is in Jy!!
    return {
        'flux': flux * conversion_factor,
        'flux_error': total_flux_error * conversion_factor,
        'background_flux': background_flux * conversion_factor,
        'median_error': median_error * conversion_factor,
        'pixel_count': n_pix
    }

# --- Main Loop ---

def perform_photometry(cutout_files, aperture_table, output_folder, psf_data=None):
    """
    Main function to perform photometry on a list of cutout files.
    
    Parameters
    ----------
    cutout_files : list
        List of paths to cutout FITS files
    aperture_table : str
        Path to CSV table with aperture parameters
    output_folder : str
        Path to output folder
    psf_dir : str, optional
        Directory containing PSF files
    """
    results = []
    
    for fits_path in cutout_files:
        # Extract ID and filter from filename
        fits_name = os.path.basename(fits_path)
        galaxy_id = fits_name.split('_')[0]
        filter_name = fits_name.split('_')[1]
        
        print(f'Processing galaxy {galaxy_id} with filter {filter_name}...')
        
        # Load image data
        with fits.open(fits_path) as hdul:
            image_data = hdul['SCI'].data if 'SCI' in hdul else hdul[1].data
            image_error = hdul['ERR'].data if 'ERR' in hdul else hdul[2].data
            
        # Get aperture parameters
        aperture_params = get_aperture_params(galaxy_id, aperture_table)
        
        # Set sigma-clipping threshold based on galaxy ID and filter
        sigma = 2.8  # Default value
        
        # Special cases for certain galaxies
        if galaxy_id in ['12332', '12282', '10314', '12164', '18332', '21452', '21477', 
                        '21541', '22606', '10592', '11136', '11142', '11420', '11451', 
                        '11494', '11716', '13103', '16419', '19042']:
            sigma = 2.0
            
        # Additional adjustments for F770W filter
        if filter_name == 'F770W' and galaxy_id in ['7136', '7904', '7922', '8469', '11716', 
                                                   '16424', '17000', '17669', '11137']:
            sigma = 2.0
        
        # Setup paths for visualisation
        #vis_path = os.path.join(output_folder, 'mosaic_fits')
        #os.makedirs(vis_path, exist_ok=True)
        
        fig_path = os.path.join(output_folder, 'mosaic_plots')
        os.makedirs(fig_path, exist_ok=True)
        
        # Estimate background with 2D-plane fit
        background_median, background_std = estimate_background(
            galaxy_id, 
            filter_name, 
            image_data, 
            aperture_params,
            sigma=sigma, 
            annulus_factor=3.0, 
            fig_path=fig_path
        )                                                                   
        
        # Measure flux
        flux_measurements = measure_flux(
            image_data, 
            image_error,
            background_median,  
            background_std, 
            aperture_params
        )

        # Get PSF and calculate aperture correction
        correction_factor = calculate_aperture_correction(psf_data, aperture_params)

        # Apply aperture correction
        corrected_flux = flux_measurements['flux'] * correction_factor
        corrected_flux_error = flux_measurements['flux_error'] * correction_factor
        corrected_background_flux = flux_measurements['background_flux'] * correction_factor
        corrected_background_error = background_std * correction_factor
        
        # --- Convert fluxes into AB magnitudes ---
        if corrected_flux > 0:
            ab_mag = -2.5 * np.log10(corrected_flux) + 8.90
        else: ab_mag = np.nan
        
        # Append results
        results.append({
            'ID': int(galaxy_id),
            'Flux': corrected_flux,
            'Flux_Err': corrected_flux_error,
            'Image_Err': flux_measurements['median_error'] * correction_factor,
            'Flux_BKG': corrected_background_flux,
            'Flux_BKG_Err': corrected_background_error,
            'AB_Mag': ab_mag,
            'N_PIX': flux_measurements['pixel_count'],
            'Apr_A': aperture_params['a'] * 2,  # Convert back to diameter for output
            'Apr_B': aperture_params['b'] * 2,  # Convert back to diameter for output
            'Apr_Xcenter': aperture_params['x_center'],
            'Apr_Ycenter': aperture_params['y_center'],
            'Apr_Theta': (aperture_params['theta'] * u.rad).to_value(u.deg)  # Convert to degrees for output
        })
    
    # Save to output table (assuming it's a pandas DataFrame)
    os.makedirs(os.path.join(output_folder, 'results'), exist_ok=True)
    output_path = os.path.join(output_folder, f'results/photometry_table_{filter_name}.csv')
    output_df = pd.DataFrame(results)
    output_df.to_csv(output_path, index=False)
    
def combine_figures(fig_path='/home/bpc/University/master/Red_Cardinal/photometry/mosaic_plots/'):
    """Function that scans a directory for plots in different filters and
       combines them if available.   
    """
    print(f"Scanning {fig_path} for galaxy images to combine...")

    # Get all F1800W images
    f1800w_pngs = glob.glob(os.path.join(fig_path, '*_F1800W.png'))

    # Track how many images we've combined
    combined_count = 0

    for f1800w_png in f1800w_pngs:
        # Extract galaxy ID from filename
        galaxy_id = os.path.basename(f1800w_png).replace('_F1800W.png', '')
        f770w_png = os.path.join(fig_path, f'{galaxy_id}.png')
        
        # Check if the standard file exists
        if os.path.exists(f770w_png):
            try:
                # Open both images
                img_f770w = Image.open(f770w_png)
                img_f1800w = Image.open(f1800w_png)
                
                # Get dimensions
                width1, height1 = img_f770w.size
                width2, height2 = img_f1800w.size
                
                # Create a new image with enough width for both images
                combined_width = width1 + width2
                combined_height = max(height1, height2)
                combined_img = Image.new('RGB', (combined_width, combined_height), (255, 255, 255))
                
                # Paste both images
                combined_img.paste(img_f770w, (0, 0))
                combined_img.paste(img_f1800w, (width1, 0))
                
                # Save combined image
                save_png = os.path.join(fig_path, f'{galaxy_id}.png')
                combined_img.save(save_png)
                
                # Delete the F1800W file
                os.remove(f1800w_png)
                
                combined_count += 1
                print(f"Combined images for galaxy {galaxy_id}")
                
            except Exception as e:
                print(f"Error combining images for galaxy {galaxy_id}: {e}")

    print(f"Combined {combined_count} galaxy image pairs.")

    # Check if any F1800W images remain (no matching F770W image)
    remaining_f1800w = glob.glob(os.path.join(fig_path, '*_F1800W.png'))
    if remaining_f1800w:
        print(f"Note: {len(remaining_f1800w)} F1800W images have no matching standard image.")

In case the code should be tested

In [ ]:
test_ids = ['8465', '7922', '9871', '12202', '8843', '7904', '8338', '10021', '10245', '11136', '12340', '20397']

In [ ]:
# --- Parameters ---
cutouts_folder = "/home/bpc/University/master/Red_Cardinal/cutouts_phot/"
output_folder = '/home/bpc/University/master/Red_Cardinal/photometry/'
aperture_table = '/home/bpc/University/master/Red_Cardinal/photometry/aperture_table_rot.csv'
fig_path = '/home/bpc/University/master/Red_Cardinal/photometry/Plots_MIRI_phot/'
os.makedirs(output_folder, exist_ok=True)

# Get all possible F770W files
all_f770w_files = glob.glob(os.path.join(cutouts_folder, f'*F770W*.fits'))

# Group F770W files by galaxy ID and filter
f770w_files = []
galaxy_ids = set([os.path.basename(f).split('_')[0] for f in all_f770w_files])

for galaxy_id in galaxy_ids:
    # Find all F770W files for this galaxy ID
    matching_files = [f for f in all_f770w_files if os.path.basename(f).startswith(galaxy_id)]
    
    # Handle special case for galaxy 11853
    if galaxy_id == '11853':
        # Use the cweb2 file if available
        cweb2_files = [f for f in matching_files if 'cweb2' in f.lower()]
        if cweb2_files:
            f770w_files.append(cweb2_files[0])
            continue  # Skip to the next galaxy
    
    # Prioritise PRIMER over COSMOS-Web
    primer_files = [f for f in matching_files if 'primer' in f.lower()]
    cweb_files = [f for f in matching_files if 'cweb' in f.lower()]
    
    if primer_files:
        f770w_files.append(primer_files[0])  # Prefer PRIMER file
    elif cweb_files:
        f770w_files.append(cweb_files[0])  # Use CWEB only if no PRIMER available

# Get all F1800W files
f1800w_files = glob.glob(os.path.join(cutouts_folder, f'*F1800W*.fits'))

psf_f770w = get_psf('F770W')
psf_f1800w = get_psf('F1800W')

perform_photometry(f770w_files, aperture_table, output_folder, psf_f770w)
perform_photometry(f1800w_files, aperture_table, output_folder, psf_f1800w)

combine_figures(fig_path)

Produce the table with the photometry

Check the contents of the original FITS files:

In [ ]:
fname = './../cutouts/7102_F770W_cutout_cweb1.fits'
#with fits.open(fname) as hdul:
    #hdul.info()


fname = './../MIRI/PRIMER_003/jw01837-o003_t003_miri_f770w_i2d.fits'
with fits.open(fname) as hdul:
    hdul.info()

fname = './../MIRI_shifted/PRIMER_003_shifted/jw01837-o003_t003_miri_f770w_i2d_shifted.fits'
with fits.open(fname) as hdul:
    hdul.info()
              
    
fname = './../photometry/mosaic_fits/11136_F770W.fits'
with fits.open(fname) as hdul:
    hdul.info()
